In [1]:
import psycopg2
import pymysql
import os
import pandas as pd
import json
from datetime import date
import re
import traceback
from decimal import Decimal, ROUND_HALF_UP
from openai import OpenAI
import Levenshtein  # 使用 Levenshtein 库来计算字符串距离
import time
from utils import get_translate_column_names,get_sql_results_json,default_converter

In [2]:
connection = psycopg2.connect(
    dbname="fdc_dc",
    user="dws_user_hwai",
    password="NewHope#1982@",
    host="124.70.57.67",
    port="8000",
)
connection.set_client_encoding('UTF8')

In [3]:
sql_query = "SELECT * FROM fdc_dwd.dwd_trade_roomsign_a_min WHERE projname = '成都锦官阁' LIMIT 15;"

In [14]:
cursor = connection.cursor()
start_time = time.time()
cursor.execute(sql_query)

results = cursor.fetchall()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"查询耗时{elapsed_time}秒")

查询耗时1.6425514221191406秒


In [17]:
column_names = cursor.description

In [18]:
column_names

(Column(name='roomcode', type_code=1043),
 Column(name='roomname', type_code=1043),
 Column(name='roomfullnm', type_code=1043),
 Column(name='buildcode', type_code=1043),
 Column(name='buildname', type_code=1043),
 Column(name='buildfullnm', type_code=1043),
 Column(name='stagecode', type_code=1043),
 Column(name='stagename', type_code=1043),
 Column(name='projcode', type_code=1043),
 Column(name='projname', type_code=1043),
 Column(name='formatcode', type_code=1043),
 Column(name='formatfullnm', type_code=1043),
 Column(name='formatname', type_code=1043),
 Column(name='citycode', type_code=1043),
 Column(name='cityname', type_code=1043),
 Column(name='corpcompanycode', type_code=1043),
 Column(name='corpcompanyname', type_code=1043),
 Column(name='propertyconsultantid', type_code=1043),
 Column(name='propertyconsultant', type_code=1043),
 Column(name='contrcode', type_code=1043),
 Column(name='contract', type_code=1043),
 Column(name='saleslipcode', type_code=1043),
 Column(name='visi

In [ ]:
cursor.close()

In [ ]:
def dws_connect(sql_query):
    dws_connect_dict = {}
    connection = psycopg2.connect(
        dbname="fdc_dc",
        user="dws_user_hwai",
        password="NewHope#1982@",
        host="124.70.57.67",
        port="8000",
    )
    connection.set_client_encoding('UTF8')
    print("连接成功")

    try:
        with connection.cursor() as cursor:
            start_time = time.time()
            cursor.execute(sql_query)

            results = cursor.fetchall()
            end_time = time.time()
            elapsed_time = end_time - start_time
            print(f"查询耗时{elapsed_time}秒")
            column_names = [desc[0] for desc in cursor.description]
            # my_df = pd.read_sql(sql_query, connection)
            # my_df.to_csv("data.csv", index=False)

            if len(results) > 100:
                dws_connect_dict["is_long"] = True
                # sql_results = json.dumps(results[:50], ensure_ascii=False, default=default_converter)
            else:
                dws_connect_dict["is_long"] = False
                sql_results = json.dumps(
                    results, ensure_ascii=False, default=default_converter
                )
                dws_connect_dict["sql_results"] = sql_results
            translate_column_names = get_translate_column_names(column_names)
            sql_results_json = get_sql_results_json(translate_column_names, results)
            dws_connect_dict["status"] = 1
            dws_connect_dict["sql_results_json"] = sql_results_json
    except Exception as e:
        traceback.print_exc()
        dws_connect_dict["status"] = 0

        error_message = str(e)
        dws_connect_dict["error_message"] = error_message
        # 获取报错信息

        print(f"SQL 执行报错: {error_message}")
    finally:
        connection.close()

    return dws_connect_dict